In [199]:
# for object questions
film_pth = '/data/c/zhuowan/Clevr/film/analysis/film_predictions.json'
# film_pth = '/data/c/zhuowan/Clevr/film/analysis/film_cnnlstm_predictions.json'
nscl_pth = '/home/zhuowan/zhuowan_ccvl19/Clevr/NSCL-PyTorch-Release/analysis/nscl_predictions.json'
mdetr_pth = '/home/zhuowan/zhuowan_ccvl19/Clevr/mdetr/analysis/mdetr_predictions.json'
question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/sliced/superCLEVR_questions_test_50k.json'

question_types = {'query': ['query_shape', 'query_size', 'query_color', 'query_material'],
                 'equal': ['equal_shape', 'equal_size', 'equal_color', 'equal_material'],
                 'compare': ['equal_integer', 'greater_than', 'less_than'],
                 'count': ['count'],
                 'exist': ['exist']}

In [220]:
# for part questions
# film_pth = '/data/c/zhuowan/Clevr/film/analysis/film_part_predictions.json'
film_pth = '/data/c/zhuowan/Clevr/film/analysis/film_lstm_part_predictions.json'
nscl_pth = '/home/zhuowan/zhuowan_ccvl19/Clevr/NSCL-PyTorch-Release/analysis/nscl_part_predictions.json'
mdetr_pth = '/home/zhuowan/zhuowan_ccvl19/Clevr/mdetr/analysis/mdetr_part_predictions.json'
question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/sliced/superCLEVR_questions_test_part_50k.json'

question_types = {'query': ['query_shape', 'query_color', 'query_material'],
                 'partquery': ['partquery_partname', 'partquery_color', 'partquery_material']}

In [221]:
import json
prediction_jsons = []
for pth in [film_pth, nscl_pth, mdetr_pth]:
    prediction_jsons.append(json.load(open(pth, 'r')))
    print(len(prediction_jsons[-1]))
assert(len(prediction_jsons[0]) == len(prediction_jsons[1]))
assert(len(prediction_jsons[2]) == len(prediction_jsons[1]))
orig_predictions = []
for i in range(len(prediction_jsons[0])):
    orig_predictions.append({'film':prediction_jsons[0][i]['prediction'],
                      'nscl':prediction_jsons[1][i]['prediction'],
                      'mdetr':prediction_jsons[2][i]['prediction']})
    
questions = json.load(open(question_pth, 'r'))['questions']
for i,q in enumerate(questions):
    orig_predictions[i]['answer'] = q['answer']
    orig_predictions[i]['question'] = q['question']
    orig_predictions[i]['image_filename'] = q['image_filename']
    orig_predictions[i]['type'] = q['program'][-1]['type']

50000
50000
50000


In [222]:
from copy import deepcopy
predictions = deepcopy(orig_predictions)
for p in predictions:
    if type(p['answer']) == bool:
        p['nscl'] = {1: True, 0:False}.get(p['nscl'], p['nscl'])
        p['film'] = {'yes':True, 'no':False}.get(p['film'], p['film'])
        p['mdetr'] = {'yes':True, 'no':False}.get(p['mdetr'], p['mdetr'])
    elif type(p['answer']) == str:
        # object questions
#         p['film'] = p['film'].split('_')[0]
#         p['mdetr'] = p['mdetr'].split(' ')[0]
#         p['answer'] = p['answer'].split(' ')[0]
        # part questions
        p['film'] = p['film'].replace('_', ' ')
        p['nscl'] = p['nscl'].replace('_', ' ')
        p['mdetr'] = p['mdetr'].replace('_', ' ')
    elif type(p['answer']) == int:
        for k in ['film', 'nscl', 'mdetr', 'answer']:
            p[k] = str(p[k])
    else:
        pass

In [223]:
for i in range(30,100):
    print(predictions[i]['film'], ',', predictions[i]['answer'])

red , brown
small , front wheel
rubber , rubber
red , green
small , airliner
red , yellow
small , back right door
rubber , metal
rubber , rubber
red , blue
small , back wheel
rubber , metal
red , red
small , minivan
red , cyan
small , back fender
rubber , rubber
rubber , metal
red , yellow
small , roof
rubber , rubber
red , green
small , fighter
red , red
small , roof
rubber , rubber
rubber , metal
red , yellow
small , front right door
rubber , metal
red , red
small , suv
red , gray
small , back right door
rubber , rubber
rubber , rubber
red , brown
small , fork
rubber , metal
red , red
small , road bike
red , brown
small , right tail light
rubber , rubber
rubber , rubber
red , yellow
small , back cover
rubber , metal
red , cyan
small , articulated bus
red , purple
small , left engine
rubber , metal
rubber , metal
red , purple
small , back left wheel
rubber , metal
red , gray
small , chopper
red , brown
small , roof
rubber , metal
rubber , rubber
red , cyan
small , trunk
rubber , metal

In [202]:
counter = {}
for st in question_types:
    for t in question_types[st]: 
        counter[t] = {'film':0, 'nscl':0, 'mdetr':0, 'answer':0}
counter['all'] = {'film':0, 'nscl':0, 'mdetr':0, 'answer':0}

for p in predictions:
    for k in ['film', 'nscl', 'mdetr']:
        if p[k] == p['answer']:
            counter[p['type']][k] += 1
            counter['all'][k] += 1
    counter[p['type']]['answer'] += 1
    counter['all']['answer'] += 1


In [203]:
for t in counter:
    print(t+',',', '.join(['%.2f' % (float(counter[t][k])/counter[t]['answer']*100) for k in ['film', 'nscl', 'mdetr']]))
#     print(t, [(counter[t][k], counter[t]['answer']) for k in ['film', 'nscl', 'mdetr']])

query_shape, 14.60, 80.67, 80.85
query_size, 56.32, 95.83, 97.10
query_color, 27.19, 88.42, 86.29
query_material, 54.08, 90.60, 91.76
equal_shape, 52.72, 86.96, 79.96
equal_size, 51.65, 95.54, 69.73
equal_color, 53.21, 91.25, 63.48
equal_material, 52.54, 86.61, 48.66
equal_integer, 54.66, 91.44, 85.93
greater_than, 68.37, 94.01, 92.62
less_than, 75.29, 93.85, 93.97
count, 47.67, 85.09, 79.23
exist, 62.08, 92.43, 88.53
all, 48.87, 89.09, 82.65


In [98]:
# random and freq baseline

from collections import Counter
all_question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/questions/superCLEVR_questions_merged.json'
all_question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/questions/superCLEVR_questions_part_partquery.json'

question_types = {'query': ['query_shape', 'query_size', 'query_color', 'query_material'],
                 'equal': ['equal_shape', 'equal_size', 'equal_color', 'equal_material'],
                 'compare': ['equal_integer', 'greater_than', 'less_than'],
                 'count': ['count'],
                 'exist': ['exist']}
question_types = {'query': ['query_shape', 'query_color', 'query_material'],
                 'partquery': ['partquery_partname', 'partquery_color', 'partquery_material']}

# all_questions = json.load(open(all_question_pth, 'r'))['questions']

answer_set = {}
for st in question_types:
    for t in question_types[st]: 
        answer_set[t] = Counter()
        
for q in all_questions:
    q_type = q['program'][-1]['type']
    answer_set[q_type][q['answer']] += 1

In [100]:
import random 

# question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/sliced/superCLEVR_questions_test_50k.json'
question_pth = '/data/c/zhuowan/SuperClevr/super-clevr/output/ver_mask/sliced/superCLEVR_questions_test_part_50k.json'
questions = json.load(open(question_pth, 'r'))['questions']
for q in questions:
    q_type = q['program'][-1]['type']
    ans = answer_set[q_type].most_common()[0][0]
    q['freq_answer'] = ans
    ans = random.choice(list(answer_set[q_type].keys()))
    q['random_answer'] = ans
    q['type'] = q_type
    
counter = {}
for st in question_types:
    for t in question_types[st]: 
        counter[t] = {'freq_answer':0, 'random_answer':0, 'answer':0}
counter['all'] = {'freq_answer':0, 'random_answer':0, 'answer':0}

for p in questions:
    for k in ['freq_answer', 'random_answer']:
        if p[k] == p['answer']:
            counter[p['type']][k] += 1
            counter['all'][k] += 1
    counter[p['type']]['answer'] += 1
    counter['all']['answer'] += 1
    

In [101]:
for t in counter:
    print(t+',',', '.join(['%.2f' % (float(counter[t][k])/counter[t]['answer']*100) for k in ['freq_answer', 'random_answer']]))
#     print(t, [(counter[t][k], counter[t]['answer']) for k in ['film', 'nscl', 'mdetr']])

query_shape, 4.95, 4.64
query_color, 12.99, 12.43
query_material, 50.15, 49.00
partquery_partname, 5.34, 1.49
partquery_color, 12.91, 12.31
partquery_material, 50.15, 49.44
all, 22.75, 21.38


In [241]:
# human annotation


obj_question_json_pth = '/data/c/zhuowan/SuperClevr/super-clevr/human_study/data/ver_mask/questions/superCLEVR_questions_merged.json'
prt_question_json_pth = '/data/c/zhuowan/SuperClevr/super-clevr/human_study/data/ver_mask/questions/superCLEVR_questions_part_partquery.json'
info_obj_pth = '/data/c/zhuowan/SuperClevr/super-clevr/human_study/data/obj_questions.json'
info_prt_pth = '/data/c/zhuowan/SuperClevr/super-clevr/human_study/data/prt_questions.json'
anno_json_pth = '/data/c/zhuowan/SuperClevr/super-clevr/human_study/data/human_results.json'
num_examples = 1000

obj_questions = json.load(open(obj_question_json_pth, 'r'))['questions']
prt_questions = json.load(open(prt_question_json_pth, 'r'))['questions']
info_obj = json.load(open(info_obj_pth, 'r'))
info_prt = json.load(open(info_prt_pth, 'r'))

N_O = 2
N_P = 1

obj_human_questions = []
prt_human_questions = []
for num in range(num_examples):
    for no in range(N_O):
        id = N_O*num + no
        q = obj_questions[info_obj['all_question_ids'][id]]
        assert(q['answer'] == info_obj['answers'][id])
        obj_human_questions.append(q)
        prt_human_questions.append({})
    for np in range(N_P):
        id = N_P*num + np
        q = prt_questions[info_prt['all_question_ids'][id]]
        assert(q['answer'] == info_prt['answers'][id])
        obj_human_questions.append({})
        prt_human_questions.append(q)    

In [242]:
anno_results = json.load(open(anno_json_pth, 'r'))
predictions = []
for anno in anno_results:
    if anno['class'][0]['taxonomy'][0][0] == 'Can\'t tell':
        human_ans = '-1'
    else:
        human_ans = anno['class'][0]['taxonomy'][0][-1].split(':')[-1].strip()
    anno_id = anno['id']
    if anno_id % 3 != 0:
        human_question = obj_human_questions[anno_id-1]
        is_part = 'obj'
    else:
        human_question = prt_human_questions[anno_id-1]
        is_part = 'prt'
    q_type = human_question['program'][-1]['type']
    answer = str(human_question['answer'])
    answer = {'True': 'yes', 'False': 'no'}.get(answer, answer)
    q = human_question['question']
    image = human_question['image']
    predictions.append({'human': str(human_ans), 'type': q_type, 'answer':answer, 'is_part':is_part, 'question':q, 'image':image})


obj_question_types = {'query': ['query_shape', 'query_size', 'query_color', 'query_material'],
                 'equal': ['equal_shape', 'equal_size', 'equal_color', 'equal_material'],
                 'compare': ['equal_integer', 'greater_than', 'less_than'],
                 'count': ['count'],
                 'exist': ['exist']}
prt_question_types = {'query': ['query_shape', 'query_color', 'query_material'],
                 'partquery': ['partquery_partname', 'partquery_color', 'partquery_material']}
      
for question_types, is_part in zip([obj_question_types, prt_question_types], ['obj', 'prt']):
    counter = {}
    for st in question_types:
        for t in question_types[st]: 
            counter[t] = {'human':0, 'no_idea':0, 'answer':0}
    counter['all'] = {'human':0, 'no_idea':0, 'answer':0}

    for p in predictions:
        if p['is_part'] != is_part:
            continue
        for k in ['human']:
    #         if p['type'] == 'count':
    #             print(p[k],p['answer'])
            if p[k] == p['answer']:
                counter[p['type']][k] += 1
                counter['all'][k] += 1
            elif p[k] == '-1':
                counter[p['type']]['no_idea'] += 1
                counter['all']['no_idea'] += 1
        counter[p['type']]['answer'] += 1
        counter['all']['answer'] += 1    

    for t in counter:
        correct_rate = float(counter[t]['human'])/counter[t]['answer']*100
        noidea_rate = float(counter[t]['no_idea'])/counter[t]['answer']*100
        print('%20s, %6.2f, \t %6.2f, \t %d, \t %d, \t %d' % (t, correct_rate, noidea_rate, counter[t]['human'], counter[t]['no_idea'], counter[t]['answer']))

    print('\n\n')


         query_shape,  63.16, 	  21.05, 	 12, 	 4, 	 19
          query_size,  70.83, 	  16.67, 	 17, 	 4, 	 24
         query_color,  65.38, 	   7.69, 	 17, 	 2, 	 26
      query_material,  78.26, 	   4.35, 	 18, 	 1, 	 23
         equal_shape,  84.21, 	   0.00, 	 16, 	 0, 	 19
          equal_size,  68.75, 	   6.25, 	 11, 	 1, 	 16
         equal_color,  66.67, 	  16.67, 	 8, 	 2, 	 12
      equal_material,  57.14, 	   0.00, 	 8, 	 0, 	 14
       equal_integer,  20.00, 	   0.00, 	 1, 	 0, 	 5
        greater_than,  75.00, 	   0.00, 	 9, 	 0, 	 12
           less_than,  50.00, 	  25.00, 	 2, 	 1, 	 4
               count,  57.14, 	   0.00, 	 36, 	 0, 	 63
               exist,  70.21, 	   6.38, 	 33, 	 3, 	 47
                 all,  66.20, 	   6.34, 	 188, 	 18, 	 284



         query_shape,  83.33, 	   0.00, 	 20, 	 0, 	 24
         query_color,  75.00, 	   0.00, 	 12, 	 0, 	 16
      query_material,  71.43, 	   0.00, 	 10, 	 0, 	 14
  partquery_partname,  37.84, 	  18.92, 	 14, 	 7

In [243]:
predictions[0]

{'human': 'metal',
 'type': 'query_material',
 'answer': 'metal',
 'is_part': 'prt',
 'question': 'What is the material of the blue thing that has a yellow tail light?',
 'image': 'superCLEVR_new_014740'}

In [244]:
with open('human_results.json','w') as f:
    json.dump(predictions, f)